In [1]:
# from pydantic import BaseModel
import pickle
import transformers
# transformers.logging.set_verbosity_info()
from newsqa import NewsQaModel, getprediction
from transformers import BertTokenizer
model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'
tokenzer = BertTokenizer.from_pretrained(model_name)
newsqa_model = NewsQaModel()
newsqa_model.load('data/bert_model.pt')




In [2]:
import sys
sys.path.append("../")

import warnings
warnings.filterwarnings("ignore")

import random
import re
import utils
import os.path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

from sklearn.feature_extraction.text import CountVectorizer
from newsqa import NewsQaExample, NewsQaModel, create_dataset, getprediction

from transformers import BertTokenizer, BertForQuestionAnswering
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)


from sklearn.metrics.pairwise import cosine_similarity
import spacy

from spacy.lang.en import English
import en_core_web_md
nlp = en_core_web_md.load()
en = English()

In [5]:
Paragraph = '''
Whenever I write about the ascent of
 modern RISC processors such as Arm and RISC-V, I always get comments about how RISC processors failed to take on Intel in the past and that they are doomed to fail again.
It is true that there was a battle between RISC and CISC processors in the 1990s. At the time workstations from companies such as Sun, Silicon Graphics and Acorn were hot stuff. Computers magazines had lots of stories about them. I remember going to conferences back then and seeing mostly Unix workstations rather than Windows machines. However, within a relatively few years these Unix workstations were pretty much all gone.
The question people cannot stop asking is: Why would history not repeat itself? Will x86 not always end up victorious in the end?
To understand why this will not happen this time around, we need to understand why RISC workstations lost back in the day and why then everything will be different this time around.
Intel’s Volume Advantage in the 90s
Production volume matters a lot when it comes to the cost of chips. Large and advanced independent chip manufacturers such as Taiwan Semiconductor Manufacturing Company (TSMC), which makes chips for Apple and many others today, did not exist in the 90s.
The one who had the most advanced manufacturing capability and the highest volume was Intel. RISC processors at the time were cheaper to design as they were simpler. However, lower volume meant higher unit prices.
Intel had the opposite situation. They had much higher design costs as the x86 instruction set made chips harder to design. However, due to their volume, they could produce chips at a lower cost per unit.
For high-end workstations which would be low volume, there was an obvious advantage with RISC chips. You could design higher-performance chips at a lower cost. For low-volume workstations, Intel would not be able to compete as their design cost could not be amortized across enough volume of chips.
The Unix workstations ended up with advanced custom-made hardware at every level which supported all sorts of fancy features such as hot-plugging hardware at runtime, and extremely fast memory. I think some workstations even let you add a CPU while it was running but don’t quote me on that.
Combined this produced awesome RISC machines at eye-watering prices.
Open vs Closed Platform
Closed platforms may give a better overall user experience, just look at Apple. The problem is that you end up with premium prices. All the RISC workstation vendors were like Apple: They sold premium products at premium prices.
The problem with such a business model is that the low-end competitor always ends up improving and closing in on you. The low-end has the benefit of volume which means high-end products very often lose out to low-end products moving up the value chain.
The Threat of “Good Enough”
The problem for RISC-based workstations was that eventually, PCs started getting good enough. They still had flaws compared to RISC-based workstations but the PCs could offer far more software and much lower prices. Thus Intel started gradually eating into the market of RISC workstations which then had to decide whether to fight in the low-end of their business or try to race ahead at the high-end.
With higher volume and higher total profits, Intel could afford to throw a lot more money at designing chips than the RISC guys. They could also use smaller node sizes to increase performance without actually having better-designed chips than the RISC guys. This made Intel gradually erode the RISC advantage.
They never stood a chance because they began at the high-end not the low end.
Did Pentium Become More RISC-like Kill RISC Processors?
In 1996 Intel shipped the Pentium Pro (P6) and their marketing material started hyping up the chip had basically been a RISC processor which could run x86 instructions. It was sold as having an exterior that could chop up x86 instructions into RISC instructions. A lot of people will point to this transition as the real reason Intel won. But there are many problems with that story. Bob Colwell who designed the Pentium Pro at Intel actually vehemently rejects that there is anything RISC-like about the processor:
Intel’s x86’s do NOT have a RISC engine “under the hood.” They implement the x86 instruction set architecture via a decode/execution scheme relying on mapping the x86 instructions into machine operations, or sequences of machine operations for complex instructions, and those operations then find their way through the microarchitecture, obeying various rules about data dependencies and ultimately time-sequencing.
The “micro-ops” that perform this feat are over 100 bits wide, carry all sorts of odd information, cannot be directly generated by a compiler, are not necessarily single cycle. But most of all, they are a microarchitecture artifice — RISC/CISC is about the instruction set architecture.
This debate has been ongoing for a long time. I wrote a story comparing RISC and CISC getting into this very topic: What Does RISC and CISC Mean in 2020?
Joel Hruska followed up with a great article later which makes a different conclusion from mine: RISC vs. CISC Is the Wrong Lens for Comparing Modern x86, ARM CPU.
Actually, I think Joel makes the most compelling arguments, but I will not delve into those details here. One can debate this issue at length but at the end of the day, it is not that relevant. Intel found ways to combat the advantage of RISC processors but there was still no inherent technical advantage in having a CISC processor. Having an x86 instruction set did not give Intel processors a performance advantage no matter how you look at it. At the end of the day having more cash to spend on making better micro-architecture and more sophisticated manufacturing made the difference.
Why RISC Today is Totally Different From the 90s
Today the chip game is completely reverse. Arm did not start by selling high-end RISC workstations. Instead, they began by dominating the low-end chips market made up of micro-controllers, embedded devices, dumb phones, and smartphones. In 2020 25 billion Arm-based chips got shipped. In comparison, the whole PC market in 2020 was 275 million units. Think about that! That is two orders of magnitude difference.
Here is a crazy fact few seem to have picked up about the world we live in today. People are used to thinking about Apple as being the small player. Macs have about a 10% market share of the computer market. So in people’s minds, Apple is a small player. What people fail to take into account is how dominant smartphones and tablets have become. An iPhone today frequently costs significantly more than a desktop computer or laptop. You got plenty of smartphones capable of outperforming low-end PCs.
These devices have high-end CPUs in them. Many of Apple’s iPad models have M1 chips inside. The very same chip used in many of their popular desktops and laptops. Thus in terms of CPU capability, it does not make that much sense to separate these markets. So here is a simple statistic that will blow your mind: Apple shipped a total of 285 million iPhones, iPads, and Macs in 2020.

Comparing the total PC market with total units shipped by Apple with Arm chips inside
Thus there are more CPUs going into Apple products than there are CPUs going into the whole PC market. Thus being king-of-the-hill in the PC market isn’t that big of a deal. This market is what Intel owns but it simply isn’t that big of a market. Apple alone is making of CPUs than Intel.
So if you look at the whole Arm market it is clear that the idea that x86 is still the dominant player is just ludicrous. Arm already won. Intel and AMD today are where the RISC workstation guys were in the 90s. They god the high high-end while facing a high-volume competitor which is rapidly catching up.
x86 Performance Edge is Not Going to Hold
The fastest supercomputers today are built using Arm,s not x86 chips. Intel and AMD are able to outperform the chips made by Apple but only at the cost of insane power usage. That is not a sustainable strategy to maintain an edge. Apple has the clear lead in terms of performance per watt. Now Apple cares more about slim design and long battery life than Intel and AMD which both make a lot of money on delivering to data centers. Thus they are not directly competing across the whole product lineup.
But other companies such as Amazon, Ampere, and Nvidia are making Arm chips for data centers which will compete directly with AMD and Intel. Ampere has their Ampere Altra Max with 128 Arm Neoverse N1 cores. The Altra outperforms the AMD EPYC 7763 with 64 cores which are regarded as the fastest x86 CPU. Amazon has come out with their Graviton3 chip with 64 Neoverse V1 cores. Meanwhile, Nvidia has announced their Grace super-chip which will have 144 Neoverse V2 cores.

Overview of current and coming high-performance Arm processors.
It might be interesting to note the difference between the Neoverse N and V cores. The N variants are for traditional server workloads while the V variants are for high-performance computing (HPC). With HPC we typically mean scientifically oriented workloads such as machine learning, lots of matrix multiplications, and other floating-point operations.
So you can imagine you might want the Neoverse N cores for hosting databases and web servers while you would use Neoverse V for machine learning, data analysis, simulations, etc.
The important point is that Arm chips exist for all markets. Apple makes the Arm chips that can compete in the desktop and laptop consumer space. Amazon, Ampere, and Nvidia supply the Arm chips which can compete in data centers.
But x86 won in the 90s over RISC! Sorry, but the rules have changed. Arm has higher volume today and Arm chip makers have access to equal or superior manufacturing through companies such as TSMC.
What About RISC-V?
It is not just when talking about Arm chips that the x86 victory over RISC in the 1990s is mentioned. It is also a point raised whenever somebody talks about the future of RISC-V. Again exactly the same story applies to RISC-V as with Arm. RISC-V chips can in principle be built by TSCM so there is no node size advantage for Intel.
What about volume? RISC-V-like Arm is starting at the low end where volume is high. RISC-V processors are going into hard drive controllers, mice, keyboards, Bluetooth devices, IoT, and a dozen other things today. Thus RISC-V is a lot more like Arm than the UltraSPARC, MIPS, and Alpha RISC processors of the 1990s.
Of course, it will take time for RISC-V to gain the kind of popularity and performance Arm enjoys today, but there is nothing stopping RISC-V from beating x86 chips sometimes in the future. However, by the time RISC-V is competing in the high-performance space I doubt there are any x86 chips around to compete against. The future will more likely be a battle between Arm and RISC-V. But you should give RISC-V at least another 5–6 years before you can expect it to be going up against Arm.
Related Stories
If you are interested in reading more of my stories about microprocessors, CISC, and RISC you may find some of these interesting:
What the Heck is a Micro-Operation? — Micro-operations is a concept that pops up often when discussing modern RISC and CISC processors. This story demystifies what a micro-operation is.
Why Pipeline a Microprocessor? — RISC processors pioneered the use of instruction pipelines. What are they and why are they useful to increase performance?
What Does RISC and CISC Mean in 2020? — Historical comparison of RISC and CISC design and a look at their convergence and the differences that still exist.
RISC-V Vector Instructions vs ARM and x86 SIMD — Focused on comparing packed-SIMD and vector-SIMD instructions and why they exist.
ARMv9: What is the Big Deal? — Goes into more details about the SVE2 vector-SIMD instruction set added to the ARMv9 architecture.
The Genius of RISC-V Microprocessors — Nothing about SIMD but about instruction compression and macro-operation fusion in RISC-V cores.
Vector Processing on CPUs and GPUs Compared — Different ways of achieving data parallelism are compared and explored.
Why Is Apple’s M1 Chip So Fast? — Relevant when discussing RISC and CISC processors as this gets into how the fact that M1 is a RISC professor plays a significant role in explaining why it performs so well.
99


4'''

Question = "what is the name of new apple chip"

In [6]:
# Reading in the dataset
data = pd.read_csv('data/newsqa-dataset-cleaned.csv')

# Getting a sample from the dataset 1756 rows
data = data.sample(frac = 0.02, random_state = 9)
data = data.reset_index(drop = True)

data.head()

,story_id,question,answer_char_ranges,validated_answers,start_idx,end_idx
0,92257ec476bf09197a5f7d3fe4afa1fa0fa558b8,What did workers discover?,157:165|649:669|None,"{""649:669"": 2}",641,669
1,f86ec18903f4df98891df668cb48647f3f931b48,How many dollars was seized in property?,132:144|864:876|864:876,NaN,860,876
2,cd20536ba3ed67f206d5d648937177a677804789,what is the next step,None|None,NaN,-1,-1
3,3a6006a860997143e1b8ecc5764b1d303f55d3d2,Where did both play?,503:519|41:64|503:519,NaN,495,522
4,45d907378dd9b4d5b8b73243be6fb2f3f9a705bb,Where is a good spot for classic straight ahea...,"2115:2127,2132:2139|2115:2127,2132:2139",NaN,2125,2141


In [7]:
def calculate_metrics(pred_start, pred_end, true_start, true_end):
   
    # Get the overlap
    overlap = set(range(true_start, true_end)).intersection(range(pred_start, pred_end))
    overlap = len(overlap)

    # If either of them have no answer
    if true_end == 0 or pred_end == 0:
        f1_score = int(true_end == pred_end)
        is_correct = int(true_end == pred_end)
        return f1_score, is_correct
    
    # If they don't overlap at all
    if overlap == 0 or pred_start >= pred_end:
        f1_score = 0
        is_correct = 0
        return f1_score, is_correct

    # If there is an overlap, we consider it correct
    is_correct = 1

    precision = overlap / (pred_end - pred_start)
    recall = overlap / (true_end - true_start)
    f1_score = (2 * precision * recall) / (precision + recall)

    return f1_score, is_correct

In [8]:
NEWS_STORIES = utils.open_pickle('data/news_stories.pkl')
text = 0
shr = 0
count = 0
sid = []
qu = []
start = []
end = []
total_examples = len(data)

In [12]:
NEWS_STORIES[row['']]

KeyError: 'f86ec18903f4df98891df668cb48647f3f931b48'

In [16]:

# took 1 hr 19 mins
correct = 0
total_f1 = 0
pred_start =0
pred_end = 0
for idx, row in data.iterrows():
    text = NEWS_STORIES[row['story_id']]
    print(text)
    question = row['question']
    print(question+ "\n\n\n")
    sid.append(row['story_id'])
    qu.append(row['question'])
    # sentences = re.split('\n', text)
    # results = rerank.rerank(question, sentences, topn=3)
    # text = listToString(results["top_sentences"])

    # Get the predictions
    shr = getprediction(text, question,tokenzer, newsqa_model, doc_stride = 512)
    pred_start = shr[1][0][0]
    pred_end = shr[1][0][1]
    start.append(pred_start)
    end.append(pred_end)
    # pred_start, pred_end = predict_answer(text, question)
    f1, is_correct = calculate_metrics(pred_start, pred_end, row['start_idx'], row['end_idx'])
    # print(pred_start, pred_end, row['start_idx'], row['end_idx'])
    # print(NEWS_STORIES[row['story_id']])
    
    total_f1 += f1
    correct += is_correct
    
    # Print progress
    utils.drawProgressBar(idx + 1, total_examples)
    
    
    
acc = correct/total_examples
f1_score = total_f1/total_examples

(CNN) -- Vehicles began streaming across the San Francisco-Oakland Bay Bridge early Tuesday -- a day ahead of schedule -- after the completion of repairs to a crack in the structure's east span.

Commuters make the trek across the San Francisco-Oakland Bay Bridge after it reopened Tuesday.

Commuters began driving over the bridge around 6:40 a.m. PT (9:40 a.m. ET), shortly after Randy Iwasaki, director of the California Department of Transportation, announced the reopening at a news conference.

"The bridge has been inspected, and it is safer than when we closed it on Friday," Iwasaki said.

Over the weekend, crews began repairing a "significant crack" that was found on the east span of the bridge during a planned closure for another project.

The target time for reopening had been early Wednesday, but crews worked nonstop overnight to repair the eyebar beam, Iwasaki said. He thanked motorists for being patient.

"I know it's been trying. I received a few e-mail notes," the official sa

In [8]:
print("F1 score: {:.4f}".format(f1_score))
print("Accuracy: {:.4f}".format(acc))

F1 score: 0.5421
Accuracy: 0.6745


In [44]:
df = pd.DataFrame({'story':sid, 'question':qu,'start':start,'end':end})

In [45]:
df

,story,question,start,end
0,92257ec476bf09197a5f7d3fe4afa1fa0fa558b8,What did workers discover?,641,661
1,f86ec18903f4df98891df668cb48647f3f931b48,How many dollars was seized in property?,856,868
2,cd20536ba3ed67f206d5d648937177a677804789,what is the next step,0,0
3,3a6006a860997143e1b8ecc5764b1d303f55d3d2,Where did both play?,-1,27
4,45d907378dd9b4d5b8b73243be6fb2f3f9a705bb,Where is a good spot for classic straight ahea...,-1,98
...,...,...,...,...
1751,084810519b9668f3f677deec54c6284ab2810e93,What may the surveillance cameras have captured?,530,540
1752,b8b5ea3238e180f7726f12c821a4b4aad9f1495d,What country is being discussed here?,9,21
1753,131375bfec062ebf7e7a5c14762725df59c60c64,Poland has been a member of the European Union...,-1,8
1754,3e2589ccaf5cbc10b2d47020a607208755412b99,What number of militants was killed by the Pak...,3221,3234


In [12]:
df.to_csv("no_ranking.csv", index=True)

After the ranking********

In [41]:
%%capture
from reranker import Rerank
import re
model_path = "nboost/pt-tinybert-msmarco"
rerank = Rerank(model_path)

In [42]:
#took 43 min 11 sec

%%capture
correct = 0
total_f1 = 0
pred_start =0
pred_end = 0
for idx, row in data.iterrows():
    text = NEWS_STORIES[row['story_id']]
    question = row['question']
    sid.append(row['story_id'])
    qu.append(row['question'])
    sentences = re.split('\n', text)
    results = rerank.rerank(question, sentences, topn=3)
    text = listToString(results["top_sentences"])

    # Get the predictions
    shr = getprediction(text, question,tokenzer, newsqa_model, doc_stride = 512)

    ans = shr[0][0]
    res = NEWS_STORIES[row['story_id']].find(ans)
    # print(res,res+len(ans))


    pred_start = res
    pred_end = res+len(ans)
    start.append(pred_start)
    end.append(pred_end)
    # pred_start, pred_end = predict_answer(text, question)
    f1, is_correct = calculate_metrics(pred_start, pred_end, row['start_idx'], row['end_idx'])
    # print(pred_start, pred_end, row['start_idx'], row['end_idx'])
    # print(NEWS_STORIES[row['story_id']])
    
    total_f1 += f1
    correct += is_correct
    
    # Print progress
    utils.drawProgressBar(idx + 1, total_examples)
    
    
    
acc = correct/total_examples
f1_score = total_f1/total_examples

In [43]:
print("F1 score: {:.4f}".format(f1_score))
print("Accuracy: {:.4f}".format(acc))

F1 score: 0.6423
Accuracy: 0.7580


In [46]:
df.to_csv("after_ranking.csv", index=True)

In [16]:

for idx, row in data.iterrows():

    text = NEWS_STORIES[row['story_id']]
    question = row['question']

    shr = getprediction(text, question,tokenzer, newsqa_model, doc_stride = 512)
    
    break

['']

12225

In [46]:
shr = getprediction(Paragraph, Question,tokenzer, newsqa_model, doc_stride = 512)

In [25]:
shr[1][0][0]

641

In [7]:
%%capture
from reranker import Rerank
import re
model_path = "nboost/pt-tinybert-msmarco"
rerank = Rerank(model_path)


In [17]:
%%capture
sentences = re.split('\n', Paragraph)
results = rerank.rerank(Question, sentences, topn=3)

In [18]:
sentences

['',
 'Whenever I write about the ascent of',
 ' modern RISC processors such as Arm and RISC-V, I always get comments about how RISC processors failed to take on Intel in the past and that they are doomed to fail again.',
 'It is true that there was a battle between RISC and CISC processors in the 1990s. At the time workstations from companies such as Sun, Silicon Graphics and Acorn were hot stuff. Computers magazines had lots of stories about them. I remember going to conferences back then and seeing mostly Unix workstations rather than Windows machines. However, within a relatively few years these Unix workstations were pretty much all gone.',
 'The question people cannot stop asking is: Why would history not repeat itself? Will x86 not always end up victorious in the end?',
 'To understand why this will not happen this time around, we need to understand why RISC workstations lost back in the day and why then everything will be different this time around.',
 'Intel’s Volume Advantage

In [19]:
results["top_sentences"]

['These devices have high-end CPUs in them. Many of Apple’s iPad models have M1 chips inside. The very same chip used in many of their popular desktops and laptops. Thus in terms of CPU capability, it does not make that much sense to separate these markets. So here is a simple statistic that will blow your mind: Apple shipped a total of 285 million iPhones, iPads, and Macs in 2020.',
 'The fastest supercomputers today are built using Arm,s not x86 chips. Intel and AMD are able to outperform the chips made by Apple but only at the cost of insane power usage. That is not a sustainable strategy to maintain an edge. Apple has the clear lead in terms of performance per watt. Now Apple cares more about slim design and long battery life than Intel and AMD which both make a lot of money on delivering to data centers. Thus they are not directly competing across the whole product lineup.',
 'Here is a crazy fact few seem to have picked up about the world we live in today. People are used to thin

In [20]:
def PrintResults(obj):
    if isinstance(obj, dict):
        for k, v in obj.items():
            if hasattr(v, '__iter__'):
                print (k)
                PrintResults(v)
            else:
                print ('%s : %s' % (k, v))
    elif isinstance(obj, list):
        for v in obj:
            if hasattr(v, '__iter__'):
                PrintResults(v)
            else:
                print(v) 
    else:
        print (obj)
def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += ele  
    
    # return string  
    return str1

In [22]:
query = results['query']
str = listToString(results['top_sentences'])

In [39]:
s = getprediction(str, query,tokenzer, newsqa_model, doc_stride = 512)
res = Paragraph.find('M1 chips ')
print('start =' ,res, ' end =' ,res+5 )

start = 6863  end = 6868


In [36]:
ans = s[0][0]
res = Paragraph.find(ans)
print(res,res+len(ans))

6863 6872


In [40]:
ans

'M1 chips '